In [67]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [68]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [69]:
data_df = pd.read_csv(r"D:\Datasets\Kaggle\Rice_type_classification\riceClassification.csv")
data_df.head()

,id,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,1,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,4,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,5,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [70]:
data_df.dropna(inplace=True)
data_df.drop(['id'], axis=1, inplace=True)
print(data_df.shape)

(18185, 11)


In [71]:
print(data_df['Class'].unique())

[1 0]


In [72]:
print(data_df['Class'].value_counts())

Class
1    9985
0    8200
Name: count, dtype: int64


In [73]:
#normalizign the dataset
original_df = data_df.copy()
for column in data_df.columns:
    data_df[column] = data_df[column]/data_df[column].abs().max()
data_df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,0.444368,0.503404,0.775435,0.744658,0.424873,0.666610,0.741661,0.537029,0.844997,0.368316,1.0
1,0.281293,0.407681,0.622653,0.750489,0.273892,0.530370,0.804230,0.409661,0.919215,0.371471,1.0
2,0.298531,0.416421,0.630442,0.756341,0.284520,0.546380,0.856278,0.412994,0.959862,0.374747,1.0
3,0.300979,0.420463,0.629049,0.764024,0.286791,0.548616,0.883772,0.414262,0.961818,0.379222,1.0
4,0.361704,0.464626,0.682901,0.775033,0.345385,0.601418,0.867808,0.452954,0.966836,0.386007,1.0


In [74]:
X = np.array(data_df.iloc[:, :-1])
Y = np.array(data_df.iloc[:,-1])

In [75]:
#splitting the dataset
X_train, X_test, Y_train ,Y_test = train_test_split(X,Y, test_size=0.3)

In [76]:
X_test, X_val, Y_test ,Y_val = train_test_split(X_test,Y_test, test_size=0.5)

In [77]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(12729, 10)
(2728, 10)
(2728, 10)


In [78]:
#Dataset class
class dataset(Dataset):
    def __init__(self, X, Y):
        self.X = torch.tensor(X, dtype=torch.float32).to(device)
        self.Y = torch.tensor(Y, dtype=torch.float32).to(device)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index], self.Y[index]



In [79]:
#training data
training_data = dataset(X_train, Y_train)
validatiion_data = dataset(X_val, Y_val)
testing_data = dataset(X_test, Y_test)

In [80]:
#DataLoader
train_dataloader = DataLoader(training_data, batch_size = 8, shuffle = True)
validatiion_dataloader = DataLoader(validatiion_data, batch_size = 8, shuffle = True)
testing_dataloader = DataLoader(testing_data, batch_size = 8, shuffle = True)

In [81]:
for x, y in train_dataloader:
    print(x)
    print('=======')
    print(y)
    break

tensor([[0.5539, 0.8120, 0.6027, 0.9748, 0.5286, 0.7442, 0.5263, 0.6509, 0.7169,
         0.7644],
        [0.6567, 0.8950, 0.6450, 0.9783, 0.6245, 0.8104, 0.7947, 0.7158, 0.7029,
         0.7872],
        [0.6714, 0.8710, 0.6777, 0.9687, 0.6437, 0.8194, 0.5281, 0.7252, 0.7001,
         0.7292],
        [0.6015, 0.7946, 0.6647, 0.9581, 0.5737, 0.7755, 0.6154, 0.6548, 0.7694,
         0.6782],
        [0.3787, 0.5048, 0.6587, 0.8367, 0.3597, 0.6154, 0.8812, 0.4726, 0.9300,
         0.4347],
        [0.6619, 0.8487, 0.6838, 0.9638, 0.6245, 0.8136, 0.5521, 0.6923, 0.7573,
         0.7042],
        [0.5106, 0.8413, 0.5283, 0.9921, 0.4869, 0.7145, 0.5196, 0.6584, 0.6460,
         0.9034],
        [0.6072, 0.8256, 0.6411, 0.9690, 0.5786, 0.7793, 0.6986, 0.6652, 0.7526,
         0.7306]])
tensor([1., 1., 1., 1., 1., 1., 1., 1.])


In [ ]:
#Simple Model
HIDDEN_NEURONS = 10
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self). __init__()

        self.input_layer = nn.Linear(X.shape[1], HIDDEN_NEURONS)
        self.linear = nn.Linear(HIDDEN_NEURONS, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.input_layer(x)
        x = self.linear(x)
        x = self.sigmoid(x)
        return x
    

model = MyModel().to(device)




In [ ]:
#Summary from torchsummary
summary(model, (X.shape[1],))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]             110
            Linear-2                    [-1, 1]              11
           Sigmoid-3                    [-1, 1]               0
Total params: 121
Trainable params: 121
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [85]:
#Loss function and optimizier
criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr = 1e-3)

In [ ]:
#training loop
 